In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
import os
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
from itertools import product, combinations
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import georasters as gr

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Dhaka Metropolitan']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)

park_entry_point_merge = 0 # ! Be careful with siplyfying topology with this, can alter the scores quite a bit.
grid_block_size = 100 # in meters2 rounded to 100m
nn_iter = 10 # tries to find nearest nodes at route finding
#block_combinations = 100
block_network = 250000 # chunks for route finding

In [3]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = ox.geocoder.geocode_to_gdf(cities)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
print('required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)')
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)
        
        # Search for data
        products = Product(iso)
        Results = products.description_contains('people per grid-cell 2020')
        list1 = []
        for p in Results:
            prints = '%s/%s\t%s\t%s' % (p.idx, p.country_name,p.dataset_name,p.path)
            list1.append(prints)
        print('wpgpDownload download -i',iso,'--id',list1[0].split("\t")[0].split('/')[0])

print('')
blocks = []
for i in iso_countries:
    # Get the country by saved path ()
    # You can also use sys.path: path = sys.path[0] +'\\' + str('ISL').lower() + '_ppp_2020.tif'
    # The sys.path is the path Python saves the python file automatically before you
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

required WorldPoP files (execute code string(s) below by country in the Jupyter terminal)
wpgpDownload download -i BGD --id 5004

BGD extracted 0.01 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((90.32978 23.75081, 90.32981 23.75050...",23.899531,23.668077,90.508832,90.329784,328693247,relation,13663697,23.783663,90.403246,"Dhaka Metropolitan, Dhaka District, Dhaka Divi...",boundary,administrative,0.6


In [4]:
# Block 2 population grids extraction
start_time = time.time()
clips = []
grids = []
print(str(grid_block_size) + 'm resolution grids extraction')
for i in range(len(cities)):
    iso = C[bound_df['display_name'][i].rsplit(',')[-1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/(grid_block_size/100)).astype(int)
    clipped['col3'] = np.floor(clipped['col']/(grid_block_size/100)).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into block by block grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = Poly
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

100m resolution grids extraction
Dhaka Metropolitan 0.12 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat
0,0-129,"POLYGON ((90.43708 23.89958, 90.43792 23.89958...",0,129,39,45949,POINT (90.43750 23.89917),POINT (10330162.784 9365206.799),1.033016e+07,9.365207e+06
1,0-130,"POLYGON ((90.43792 23.89958, 90.43875 23.89958...",0,130,37,45950,POINT (90.43833 23.89917),POINT (10330184.751 9365411.161),1.033018e+07,9.365411e+06
2,0-131,"POLYGON ((90.43875 23.89958, 90.43958 23.89958...",0,131,33,45950,POINT (90.43917 23.89917),POINT (10330206.710 9365615.525),1.033021e+07,9.365616e+06
3,1-127,"POLYGON ((90.43542 23.89875, 90.43625 23.89875...",1,127,41,45952,POINT (90.43583 23.89833),POINT (10330341.105 9364774.166),1.033034e+07,9.364774e+06
4,1-128,"POLYGON ((90.43625 23.89875, 90.43708 23.89875...",1,128,41,45952,POINT (90.43667 23.89833),POINT (10330363.087 9364978.533),1.033036e+07,9.364979e+06
...,...,...,...,...,...,...,...,...,...,...
31312,99-95,"POLYGON ((90.40875 23.81708, 90.40958 23.81708...",99,95,8,46255,POINT (90.40917 23.81667),POINT (10351445.505 9355860.653),1.035145e+07,9.355861e+06
31313,99-96,"POLYGON ((90.40958 23.81708, 90.41042 23.81708...",99,96,8,46255,POINT (90.41000 23.81667),POINT (10351467.860 9356065.726),1.035147e+07,9.356066e+06
31314,99-97,"POLYGON ((90.41042 23.81708, 90.41125 23.81708...",99,97,6,46255,POINT (90.41083 23.81667),POINT (10351490.208 9356270.799),1.035149e+07,9.356271e+06
31315,99-98,"POLYGON ((90.41125 23.81708, 90.41208 23.81708...",99,98,6,46255,POINT (90.41167 23.81667),POINT (10351512.548 9356475.875),1.035151e+07,9.356476e+06


In [5]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
    #graphs.append(graph)
    
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    #road_nodes.append(road_node)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    #road_edges.append(road_edge)
    
    # Exclude highways and ramps on edges    
    edge_filter = road_edge[(road_edge['highway'].str.contains('motorway') | 
          (road_edge['highway'].str.contains('trunk') & 
           road_edge['maxspeed'].astype(str).str.contains(
               '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
    road_edges.append(edge_filter)
    
    # Exclude isolated nodes
    fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
    newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
    road_nodes.append(newnodes)
    
    # Get only necessary road connections columns for network performance
    road_con = edge_filter[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    
    # formatting to graph again.
    newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
    edge_filter = edge_filter.set_index(['from','to','keys'])
    edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]
    graph2 = ox.graph_from_gdfs(newnodes, edge_filter)
                            
    graphs.append(graph2)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    
road_edges[0]

Dhaka Metropolitan done 0.87 mns


,from,to,keys,osmid,oneway,lanes,name,highway,maxspeed,length,...,junction,access,ref,width,bridge,service,tunnel,area,geometry_m,key
0,60916997,3501308373,0,691209729,True,3,কাজী নজরুল ইসলাম এভিনিউ,trunk,60,14.774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10371413.627 9350430.712, 10371449...",60916997-3501308373
1,60916997,6785381353,0,723435766,False,NaN,NaN,service,NaN,202.053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10371413.627 9350430.712, 10371414...",60916997-6785381353
2,60917111,6139734068,0,24448678,True,2,মতিঝিল সড়ক,primary,NaN,91.086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10375627.766 9355967.224, 10375771...",60917111-6139734068
3,60917111,387810798,0,33851117,False,NaN,NaN,unclassified,NaN,59.390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10375627.766 9355967.224, 10375512...",60917111-387810798
4,60917472,3006503617,0,24403714,True,2,Kazi Nazrul Islam Avenue,secondary,40,57.667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10372145.354 9350296.574, 10372198...",60917472-3006503617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103755,9769610118,9433754696,0,1022891945,True,NaN,NaN,tertiary,NaN,253.325,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10342250.218 9347344.294, 10341656...",9769610118-9433754696
103756,9769610118,9433754697,0,1063596973,False,NaN,NaN,residential,NaN,340.102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10342250.218 9347344.294, 10342220...",9769610118-9433754697
103757,9769610119,9431430168,0,875692627,False,NaN,NaN,residential,NaN,51.090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10342259.826 9347372.526, 10342295...",9769610119-9431430168
103758,9769610119,9769610118,0,875692627,False,NaN,NaN,residential,NaN,12.274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (10342259.826 9347372.526, 10342250...",9769610119-9769610118


In [6]:
# Block 4 city greenspace
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()    
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    print(i, 'done')
    parks = parks.reset_index()
    parks_in_range.append(parks)
parks_in_range[0]

Dhaka Metropolitan done


,components,geometry,element_type,osmid,leisure,name,name:bn,name:en,barrier,addr:postcode,...,opening_hours:covid19,ways,type,WDPA_ID:ref,boundary,operator:type,protect_class,protection_title,geometry_w_buffer,geom buffer diff
0,0,"POLYGON ((90.42091 23.80544, 90.42017 23.80533...",way,10263606,park,Baridhara Play Ground,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.42005 23.80628, 90.42075 23.80647...","POLYGON ((90.42075 23.80647, 90.42076 23.80647..."
1,1,"POLYGON ((90.41585 23.79739, 90.41583 23.79911...",way,12493048,park,Justice Shahabuddin Park (Gulshan Park),None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.41591 23.79732, 90.41558 23.79712...","POLYGON ((90.41558 23.79712, 90.41557 23.79712..."
2,2,"POLYGON ((90.41125 23.80211, 90.41074 23.80200...",way,23852697,park,গুলশান লেক পার্ক,None,Gulshan Lake Park,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.41134 23.80214, 90.41177 23.80093...","POLYGON ((90.41177 23.80093, 90.41177 23.80092..."
3,3,"POLYGON ((90.41791 23.79968, 90.41784 23.80025...",way,23895458,park,Baridhara Park,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.41781 23.79968, 90.41774 23.80024...","POLYGON ((90.41774 23.80024, 90.41774 23.80025..."
4,4,"POLYGON ((90.41557 23.79203, 90.41575 23.79162...",way,24338550,park,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.41556 23.79212, 90.41665 23.79223...","POLYGON ((90.41665 23.79223, 90.41666 23.79223..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,163,"POLYGON ((90.41272 23.77166, 90.41302 23.77145...",way,968068212,park,Hatirjheel Park and Playground,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.41268 23.77175, 90.41460 23.77250...","POLYGON ((90.41460 23.77250, 90.41461 23.77250..."
164,165,"POLYGON ((90.38701 23.87197, 90.38702 23.87119...",way,1020268351,park,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.38701 23.87206, 90.38777 23.87207...","POLYGON ((90.38777 23.87207, 90.38778 23.87207..."
165,166,"POLYGON ((90.35844 23.84439, 90.35860 23.84442...",way,1046695638,park,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((90.35834 23.84438, 90.35829 23.84465...","POLYGON ((90.35829 23.84465, 90.35829 23.84466..."
166,167,"POLYGON ((90.42997 23.73914, 90.43018 23.73916...",relation,4790846,park,বালুর মাঠ,None,None,None,None,...,None,"[338596015, 338596016]",multipolygon,None,None,None,None,None,"POLYGON ((90.43006 23.73924, 90.43017 23.73925...","MULTIPOLYGON (((90.43017 23.73925, 90.43041 23..."


In [7]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 50 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_sqr2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_sqr3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_sqr5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Dhaka Metropolitan 0.0 % done 0.0  mns
Dhaka Metropolitan 29.8 % done 0.11  mns
Dhaka Metropolitan 59.5 % done 0.21  mns
Dhaka Metropolitan 89.3 % done 0.3  mns
Dhaka Metropolitan 100 % done 0.35  mns


,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5
0,6852209823,23.806323,90.420373,3,NaN,NaN,POINT (90.42037 23.80632),POINT (10354513.039 9358318.106),6852209823,0,...,9.358318e+06,"POLYGON ((90.42091 23.80544, 90.42017 23.80533...","POLYGON ((90.42017 23.80533, 90.42000 23.80620...",46326.261998,4.632626e+04,1.000000,1.188667,1.090260,1.059303,1.035171
1,85576605,23.798916,90.415931,3,NaN,NaN,POINT (90.41593 23.79892),POINT (10356376.552 9357008.627),85576605,1,...,9.357009e+06,"POLYGON ((90.41585 23.79739, 90.41583 23.79911...","POLYGON ((90.41518 23.79719, 90.41500 23.79727...",195014.036999,2.299870e+05,0.847935,5.003786,2.236914,1.710407,1.379939
2,85576762,23.797707,90.415928,3,NaN,NaN,POINT (90.41593 23.79771),POINT (10356700.076 9356972.725),85576762,1,...,9.356973e+06,"POLYGON ((90.41585 23.79739, 90.41583 23.79911...","POLYGON ((90.41399 23.79823, 90.41406 23.79840...",217206.259482,2.299870e+05,0.944428,5.573207,2.360764,1.772971,1.410006
3,85577410,23.797631,90.413962,3,NaN,NaN,POINT (90.41396 23.79763),POINT (10356667.913 9356486.422),85577410,1,...,9.356486e+06,"POLYGON ((90.41585 23.79739, 90.41583 23.79911...","POLYGON ((90.41523 23.79908, 90.41537 23.79896...",215700.636239,2.299870e+05,0.937882,5.534575,2.352568,1.768865,1.408046
4,5216114479,23.798239,90.413901,3,NaN,NaN,POINT (90.41390 23.79824),POINT (10356503.562 9356488.943),5216114479,1,...,9.356489e+06,"POLYGON ((90.41585 23.79739, 90.41583 23.79911...","POLYGON ((90.41576 23.79897, 90.41583 23.79880...",225936.034406,2.299870e+05,0.982386,5.797201,2.407738,1.796413,1.421162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,264880599,23.812311,90.347755,3,NaN,NaN,POINT (90.34775 23.81231),POINT (10350943.222 9340621.996),264880599,167,...,9.340622e+06,"POLYGON ((90.34767 23.81267, 90.34763 23.81270...","POLYGON ((90.34753 23.81416, 90.34773 23.81417...",213007.026865,4.714676e+06,0.045180,5.465461,2.337832,1.761471,1.404512
346,4263480801,23.823954,90.351794,3,NaN,NaN,POINT (90.35179 23.82395),POINT (10347941.348 9341962.623),4263480801,167,...,9.341963e+06,"POLYGON ((90.34767 23.81267, 90.34763 23.81270...","POLYGON ((90.35162 23.82210, 90.35143 23.82213...",333006.137971,4.714676e+06,0.070632,8.544469,2.923092,2.044380,1.535808
347,4399963731,23.817041,90.352518,1,NaN,NaN,POINT (90.35252 23.81704),POINT (10349809.298 9341934.718),4399963731,167,...,9.341935e+06,"POLYGON ((90.34767 23.81267, 90.34763 23.81270...","MULTIPOLYGON (((90.35142 23.81548, 90.35127 23...",215760.671260,4.714676e+06,0.045764,5.536115,2.352895,1.769029,1.408124
348,4476487955,23.818027,90.352294,1,NaN,NaN,POINT (90.35229 23.81803),POINT (10349539.670 9341909.080),4476487955,167,...,9.341909e+06,"POLYGON ((90.34767 23.81267, 90.34763 23.81270...","POLYGON ((90.35137 23.81638, 90.35119 23.81647...",360518.691912,4.714676e+06,0.076467,9.250403,3.041447,2.099199,1.560386


In [8]:
#ParkRoads[0]['geometry_x'].to_file('D:/Dumps/Chandigarh_nodes.shp')

In [9]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,geometry_m_buffer,components,comp_centroid,centr_dist
0,6852209823,23.806323,90.420373,3,NaN,NaN,POINT (90.42037 23.80632),POINT (10354513.039 9358318.106),6852209823,0,...,4.632626e+04,1.000000,1.188667,1.090260,1.059303,1.035171,POLYGON EMPTY,0,POINT (90.42037 23.80632),0.0
1,85576605,23.798916,90.415931,3,NaN,NaN,POINT (90.41593 23.79892),POINT (10356376.552 9357008.627),85576605,1,...,2.299870e+05,0.847935,5.003786,2.236914,1.710407,1.379939,POLYGON EMPTY,1,POINT (90.41593 23.79892),0.0
2,85576762,23.797707,90.415928,3,NaN,NaN,POINT (90.41593 23.79771),POINT (10356700.076 9356972.725),85576762,1,...,2.299870e+05,0.944428,5.573207,2.360764,1.772971,1.410006,POLYGON EMPTY,2,POINT (90.41593 23.79771),0.0
3,85577410,23.797631,90.413962,3,NaN,NaN,POINT (90.41396 23.79763),POINT (10356667.913 9356486.422),85577410,1,...,2.299870e+05,0.937882,5.534575,2.352568,1.768865,1.408046,POLYGON EMPTY,3,POINT (90.41396 23.79763),0.0
4,5216114479,23.798239,90.413901,3,NaN,NaN,POINT (90.41390 23.79824),POINT (10356503.562 9356488.943),5216114479,1,...,2.299870e+05,0.982386,5.797201,2.407738,1.796413,1.421162,POLYGON EMPTY,4,POINT (90.41390 23.79824),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,264880599,23.812311,90.347755,3,NaN,NaN,POINT (90.34775 23.81231),POINT (10350943.222 9340621.996),264880599,167,...,4.714676e+06,0.045180,5.465461,2.337832,1.761471,1.404512,POLYGON EMPTY,345,POINT (90.34775 23.81231),0.0
346,4263480801,23.823954,90.351794,3,NaN,NaN,POINT (90.35179 23.82395),POINT (10347941.348 9341962.623),4263480801,167,...,4.714676e+06,0.070632,8.544469,2.923092,2.044380,1.535808,POLYGON EMPTY,346,POINT (90.35179 23.82395),0.0
347,4399963731,23.817041,90.352518,1,NaN,NaN,POINT (90.35252 23.81704),POINT (10349809.298 9341934.718),4399963731,167,...,4.714676e+06,0.045764,5.536115,2.352895,1.769029,1.408124,POLYGON EMPTY,347,POINT (90.35252 23.81704),0.0
348,4476487955,23.818027,90.352294,1,NaN,NaN,POINT (90.35229 23.81803),POINT (10349539.670 9341909.080),4476487955,167,...,4.714676e+06,0.076467,9.250403,3.041447,2.099199,1.560386,POLYGON EMPTY,348,POINT (90.35229 23.81803),0.0


In [12]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    #blockA = block_combinations
    print(cities[l])
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    
    # Reduce the size of combinations per iteration
    len4 = 1
    len5 = len1 * len2
    blockC = len5
    while blockC > 10000000:
        blockC = len5 / len4
        #print(blockC, len4)
        len4 = len4+1
        
    # Amount of grids taken per iteration block
    block = round(len1 / len4)
    
    output = pd.DataFrame()
    len_mat = 0
    len_df = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len4):
        # Check all grid-park combinations per block
        l1, l2 = range(i*block,(i+1)*block), range(0,len2)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_sqr2','size_infl_sqr3','size_infl_sqr5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_sqr2'])
        infl3 = list(node_merged['size_infl_sqr3'])
        infl5 = list(node_merged['size_infl_sqr5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])
        len_mat = len_mat + len(mat_df)
        len_df = len_df + len(node_merged)
            
        if ( (i+1) % 1 == 0) | ((i+1) == len4): 
            print((i+1),'/',len4,'comb. done',round((time.time() - start_time) / 60,2),' mns')
            
            print('of',len_df,'within a Gravity model variant in one of',thresholds,'m threshold:',len_mat)
            len_mat = 0
            len_df = 0
    # Renaming columns
    print('total combinations within distance',len(output))
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5',
                      'parkshare_walked','park_area','walk_area_m2']
    
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_sqr2','size_infl_sqr3','size_infl_sqr5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        try:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        except:
            nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    #output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
    #                                               ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]    

Dhaka Metropolitan
1 / 3 comb. done 0.37  mns
of 3653650 within a Gravity model variant in one of [300, 600, 1000] m threshold: 21890
2 / 3 comb. done 0.76  mns
of 3653650 within a Gravity model variant in one of [300, 600, 1000] m threshold: 25070
3 / 3 comb. done 1.14  mns
of 3653650 within a Gravity model variant in one of [300, 600, 1000] m threshold: 5533
total combinations within distance 0
0.0 % gridentry done 1.14  mns
100 % gridentry done 1.22  mns


,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,...,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_sqr2,size_infl_sqr3,size_infl_sqr5,grid_osm,geometry,geometry_m
0,860,1033.801396,948.215519,975.926425,998.677102,860,0,POINT (90.41917 23.81000),POINT (10353496.889 9358127.883),0,...,POINT (90.42037 23.80632),"POLYGON ((90.42091 23.80544, 90.42017 23.80533...",POINT (10354513.039 9358318.106),46326.261998,1.090260,1.059303,1.035171,4908604274,POINT (90.41910 23.80968),POINT (10353580.171 9358101.245)
1,975,821.647942,753.625728,775.649889,793.731744,975,0,POINT (90.41917 23.80917),POINT (10353719.870 9358103.660),0,...,POINT (90.42037 23.80632),"POLYGON ((90.42091 23.80544, 90.42017 23.80533...",POINT (10354513.039 9358318.106),46326.261998,1.090260,1.059303,1.035171,4908604274,POINT (90.41910 23.80968),POINT (10353580.171 9358101.245)
2,251511,991.648340,1382.111579,1237.317912,1132.483428,975,24,POINT (90.41917 23.80917),POINT (10353719.870 9358103.660),3,...,POINT (90.41640 23.80650),"POLYGON ((90.41791 23.79968, 90.41784 23.80025...",POINT (10354358.703 9357345.202),20063.020447,0.717488,0.801450,0.875640,4908604274,POINT (90.41910 23.80968),POINT (10353580.171 9358101.245)
3,303706,996.292301,1180.473983,1115.577821,1066.239169,975,29,POINT (90.41917 23.80917),POINT (10353719.870 9358103.660),3,...,POINT (90.41684 23.80615),"POLYGON ((90.41791 23.79968, 90.41784 23.80025...",POINT (10354464.978 9357442.289),27760.536568,0.843976,0.893073,0.934399,4908604274,POINT (90.41910 23.80968),POINT (10353580.171 9358101.245)
4,861,993.979723,911.690584,938.334075,960.208404,861,0,POINT (90.42000 23.81000),POINT (10353519.172 9358333.034),0,...,POINT (90.42037 23.80632),"POLYGON ((90.42091 23.80544, 90.42017 23.80533...",POINT (10354513.039 9358318.106),46326.261998,1.090260,1.059303,1.035171,4206253691,POINT (90.41998 23.81029),POINT (10353440.573 9358335.453)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52488,3653137,3230.851825,955.814294,1434.458629,1984.904575,30804,349,POINT (90.36500 23.82000),POINT (10349359.381 9345092.651),167,...,POINT (90.35206 23.81843),"POLYGON ((90.34767 23.81267, 90.34763 23.81270...",POINT (10349425.780 9341862.481),445301.530322,3.380209,2.252314,1.627711,7867028835,POINT (90.36499 23.82003),POINT (10349350.358 9345091.073)
52489,3653251,3209.279491,949.432340,1424.880777,1971.651406,30918,349,POINT (90.36500 23.81917),POINT (10349582.180 9345067.947),167,...,POINT (90.35206 23.81843),"POLYGON ((90.34767 23.81267, 90.34763 23.81270...",POINT (10349425.780 9341862.481),445301.530322,3.380209,2.252314,1.627711,386278584,POINT (90.36527 23.81903),POINT (10349624.872 9345129.632)
52490,3653362,3203.285857,947.659185,1422.219677,1967.969160,31029,349,POINT (90.36500 23.81833),POINT (10349804.985 9345043.243),167,...,POINT (90.35206 23.81843),"POLYGON ((90.34767 23.81267, 90.34763 23.81270...",POINT (10349425.780 9341862.481),445301.530322,3.380209,2.252314,1.627711,3639300318,POINT (90.36480 23.81839),POINT (10349784.024 9344994.707)
52491,3653472,3212.959575,950.521055,1426.514690,1973.912301,31139,349,POINT (90.36500 23.81750),POINT (10350027.797 9345018.536),167,...,POINT (90.35206 23.81843),"POLYGON ((90.34767 23.81267, 90.34763 23.81270...",POINT (10349425.780 9341862.481),445301.530322,3.380209,2.252314,1.627711,3639272874,POINT (90.36501 23.81750),POINT (10350028.389 9345021.421)


In [13]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Dhaka Metropolitan 52493


In [14]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = block_network

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    1217624
    # Divide in chunks of block for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        s_mat5 = list([])
        mat_nn = []
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(-1)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
                s_mat5.append(1)
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                    s_mat5.append(0)
                except:
                    # Otherwise the nearest node is taken, which is iterated X times at max, check assumptions, block #0 
                    # Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < nn_iter and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                            s_mat4.append(str(len3)+'grid > n-park')
                            s_mat5.append(1)
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                                s_mat4.append(str(len3)+'n-park > grid')
                                s_mat5.append(0)
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                    s_mat4.append(str(len3)+'n-grid > park')
                                    s_mat5.append(1)
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                        s_mat4.append(str(len3)+'park > n-grid')
                                        s_mat5.append(0)
                                    except:
                                        if len3 == nearest_node_iterations:
                                            #print(i+block*k,i+block*k,
                                            #      'No route between grid and park-entry and their both 10 alternatives')
                                            pass
                                        pass
                    #print(len(alt_route))
                    if len(alt_route) == 0: 
                        alt = alt_route 
                    else: 
                        alt = alt_route[0]
                    len4 = len(alt)
                    #print(len4)
                    #mat_nn = []
                    if len4 > 0:
                        #print('for index',i+block*k,'nearest node found between', 
                        #                       alt[0],'and',alt[-1])
                        mat_nn.append(i+block*k)
                        s_mat.append(alt)
                        shortest_to = list(alt[1:len(alt)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')
                        s_mat5.append(2)
                        print(i+block*k,'No route between grid and park-entry and their both',nn_iter,'alternatives')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')
        print(len(mat_nn),'nearest nodes found')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # dissolve route
        routes2 = routes[['route','geometry']].dissolve('route')
        
        # get used grid- and parkosm. Differs at NN-route.
        route_reset = routes.reset_index()
        origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
        origin = origin.reset_index().iloc[:,-1]
        dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
        dest = dest.reset_index().iloc[:,-1]
        
        # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
        routes2['G1-P0'] = s_mat5
        routes2['real_grid'] = np.where(routes2['G1-P0'] == 1, origin, dest)
        routes2['real_park'] = np.where(routes2['G1-P0'] == 1, dest, origin)
        routes2['way_calc.'] = s_mat4
        
        # get route cost, steps, additional information.
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                              'Grid_m_centroid','walk_area_m2','size_infl_sqr2','size_infl_sqr3',
                                              'size_infl_sqr5','raw euclidean']],
                                                left_index = True, right_index = True)
        routes2 = pd.merge(routes2, road_nodes[j]['geometry_m'], how = 'left', left_on = 'real_grid', right_index = True)
        
        # calculate distance of used road-entry for grid-centroid.
        routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043
                                                       ).distance(routes2['geometry_m']),3)
        
        # Calculcate total route cost for the four gravity variants
        routes2['raw_Tcost'] = routes2['route_cost'] + routes2['real_G-entry']
        routes2['grav2_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr2']
        routes2['grav3_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr3']
        routes2['grav5_Tcost'] = (routes2['route_cost'] + routes2['real_G-entry']) / routes2['size_infl_sqr5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]
# 60m Philadelphia

Dhaka Metropolitan 1 / 1 range 0 - 52493
0.0 % done 0.0 mns
19.05 % done 0.06 mns
38.1 % done 0.13 mns
57.15 % done 0.21 mns
76.2 % done 0.34 mns
95.25 % done 0.41 mns
0 nearest nodes found
100.0 % pathfinding done 0.46 mns
formatting done 0.76 mns
dissolving done 1.03 mns


,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,Park_No,...,size_infl_sqr3,size_infl_sqr5,raw euclidean,geometry_m,real_G-entry,raw_Tcost,grav2_Tcost,grav3_Tcost,grav5_Tcost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6852209823,normal way,940.509,9,860,4908604274,0,...,1.059303,1.035171,1033.801396,POINT (10353580.171 9358101.245),87.438,1027.947,942.845794,970.399773,993.021614,860-0
1,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6852209823,normal way,940.509,9,975,4908604274,0,...,1.059303,1.035171,821.647942,POINT (10353580.171 9358101.245),139.719,1080.228,990.798578,1019.753943,1043.526321,975-0
2,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6387748575,normal way,1209.627,10,975,4908604274,3,...,0.801450,0.875640,991.648340,POINT (10353580.171 9358101.245),139.719,1349.346,1880.653307,1683.631090,1540.981740,975-3
3,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6389411938,normal way,1229.993,11,975,4908604274,3,...,0.893073,0.934399,996.292301,POINT (10353580.171 9358101.245),139.719,1369.712,1622.926705,1533.706852,1465.875610,975-3
4,"MULTILINESTRING ((90.41998 23.81029, 90.42003 ...",1,4206253691,6852209823,normal way,943.538,10,861,4206253691,0,...,1.059303,1.035171,993.979723,POINT (10353440.573 9358335.453),78.636,1022.174,937.550726,964.949961,987.444756,861-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52488,"MULTILINESTRING ((90.36499 23.82003, 90.36391 ...",1,7867028835,4476487961,normal way,1897.765,25,30804,7867028835,167,...,2.252314,1.627711,3230.851825,POINT (10349350.358 9345091.073),9.160,1906.925,564.144155,846.651338,1171.537527,30804-167
52489,"MULTILINESTRING ((90.36527 23.81903, 90.36467 ...",1,386278584,4476487961,normal way,1785.574,24,30918,386278584,167,...,2.252314,1.627711,3209.279491,POINT (10349624.872 9345129.632),75.018,1860.592,550.437013,826.080054,1143.072407,30918-167
52490,"MULTILINESTRING ((90.36480 23.81839, 90.36376 ...",1,3639300318,4476487961,normal way,1779.527,23,31029,3639300318,167,...,2.252314,1.627711,3203.285857,POINT (10349784.024 9344994.707),52.868,1832.395,542.095221,813.560931,1125.749312,31029-167


In [15]:
Routes[0]

,geometry,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,grid_osm,Park_No,...,size_infl_sqr3,size_infl_sqr5,raw euclidean,geometry_m,real_G-entry,raw_Tcost,grav2_Tcost,grav3_Tcost,grav5_Tcost,gridpark_no
index,,,,,,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6852209823,normal way,940.509,9,860,4908604274,0,...,1.059303,1.035171,1033.801396,POINT (10353580.171 9358101.245),87.438,1027.947,942.845794,970.399773,993.021614,860-0
1,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6852209823,normal way,940.509,9,975,4908604274,0,...,1.059303,1.035171,821.647942,POINT (10353580.171 9358101.245),139.719,1080.228,990.798578,1019.753943,1043.526321,975-0
2,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6387748575,normal way,1209.627,10,975,4908604274,3,...,0.801450,0.875640,991.648340,POINT (10353580.171 9358101.245),139.719,1349.346,1880.653307,1683.631090,1540.981740,975-3
3,"MULTILINESTRING ((90.41910 23.80968, 90.41925 ...",1,4908604274,6389411938,normal way,1229.993,11,975,4908604274,3,...,0.893073,0.934399,996.292301,POINT (10353580.171 9358101.245),139.719,1369.712,1622.926705,1533.706852,1465.875610,975-3
4,"MULTILINESTRING ((90.41998 23.81029, 90.42003 ...",1,4206253691,6852209823,normal way,943.538,10,861,4206253691,0,...,1.059303,1.035171,993.979723,POINT (10353440.573 9358335.453),78.636,1022.174,937.550726,964.949961,987.444756,861-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52488,"MULTILINESTRING ((90.36499 23.82003, 90.36391 ...",1,7867028835,4476487961,normal way,1897.765,25,30804,7867028835,167,...,2.252314,1.627711,3230.851825,POINT (10349350.358 9345091.073),9.160,1906.925,564.144155,846.651338,1171.537527,30804-167
52489,"MULTILINESTRING ((90.36527 23.81903, 90.36467 ...",1,386278584,4476487961,normal way,1785.574,24,30918,386278584,167,...,2.252314,1.627711,3209.279491,POINT (10349624.872 9345129.632),75.018,1860.592,550.437013,826.080054,1143.072407,30918-167
52490,"MULTILINESTRING ((90.36480 23.81839, 90.36376 ...",1,3639300318,4476487961,normal way,1779.527,23,31029,3639300318,167,...,2.252314,1.627711,3203.285857,POINT (10349784.024 9344994.707),52.868,1832.395,542.095221,813.560931,1125.749312,31029-167


In [16]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        Rclean = Routes[n][Routes[n]['way_calc.'] != 'no way'].reset_index()
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Rclean.iloc[Rclean.groupby('gridpark_no')[(str(l1[i]) +'_Tcost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_Tcost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['population','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_Tcost'] <= t]
            thold[score] = t - thold[l1[i] + '_Tcost']
            thold['pop' + score] = thold[score] * thold['population']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['population'] / thold['walk_area_ha' + str2]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + str2]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2].sum()
            
            gs['walkha_person_' + score] = locals()[str1].groupby('Grid_No')['walkha_person' + str2].mean()

            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['population'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/')
                
            gs.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_lines/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['population'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['population'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/')
            
        gridsc.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_geoms/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/')
        
        gridsc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/radius-euclidean adjusted popgrid access.csv')

popg_acc    


Dhaka Metropolitan
entrance 0.02 mns
grid  300
grid  600
grid  1000
gravity**(1/2) 0.99 mns
grid  300
grid  600
grid  1000
gravity**(1/3) 2.03 mns
grid  300
grid  600
grid  1000
gravity**(1/5) 3.03 mns
grid  300
grid  600
grid  1000
Dhaka Metropolitan done 4.01 mns


Dhaka Metropolitan
entrance_300        1 high              0.001281
                    2 medium            0.013622
                    3 low               0.033066
                    4 no                0.952031
entrance_600        1 high              0.008381
                    2 medium            0.044442
                    3 low               0.103462
                    4 no                0.843714
entrance_1000       1 high              0.027199
                    2 medium            0.103608
                    3 low               0.137017
                    4 no                0.732176
gravity**(1/2)_300  1 high              0.001354
                    2 medium            0.019745
                    3 low               0.053814
                    4 no                0.925086
gravity**(1/2)_600  1 high              0.013132
                    2 medium            0.077394
                    3 low               0.158596
                    4 no                0.750879
gravity**(1/2)_1000 1 high              0.068497
                    2 medium            0.150706
                    3 low               0.130795
                    4 no                0.650001
gravity**(1/3)_300  1 high              0.001105
                    2 medium            0.015673
                    3 low               0.036852
                    4 no                0.946370
gravity**(1/3)_600  1 high              0.007621
                    2 medium            0.051190
                    3 low               0.138306
                    4 no                0.802882
gravity**(1/3)_1000 1 high              0.040922
                    2 medium            0.135548
                    3 low               0.155147
                    4 no                0.668383
gravity**(1/5)_300  1 high              0.000861
                    2 medium            0.014296
                    3 low               0.031397
                    4 no                0.953446
gravity**(1/5)_600  1 high              0.006944
                    2 medium            0.043899
                    3 low               0.116910
                    4 no                0.832248
gravity**(1/5)_1000 1 high              0.027776
                    2 medium            0.116720
                    3 low               0.159313
                    4 no                0.696191

In [17]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'popreach'+score]] = str1.groupby('Park_No')[score,'pop' + score,'population'].sum()
            
            prk['walkha_person_' + score] = str1.groupby('Park_No')['walkha_person' + str2].mean()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
        
            if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/'):
                os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/')
            
            prk.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_lines/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/')
            
        prksc.to_file('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_geoms/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        
        if not os.path.exists('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/'):
            os.makedirs('D:Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/')
            
        prksc.to_csv('D:/Dumps/Scores output WP2-OSM/'+str(grid_block_size)+'m grids/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[0][0])

park 300
park 600
park 1000
entrance 0.05 mns
park 300
park 600
park 1000
gravity**(1/2) 0.4 mns
park 300
park 600
park 1000
gravity**(1/3) 0.58 mns
park 300
park 600
park 1000
gravity**(1/5) 0.66 mns
Dhaka Metropolitan done 0.66 mns


,tr_300,pop_tr_300,popreachtr_300,walkha_person_tr_300,tr_300_parks,tr_300route,tr_300Grid:Pentry,tr_300_access,tr_600,pop_tr_600,...,tr_600Grid:Pentry,tr_600_access,tr_1000,pop_tr_1000,popreachtr_1000,walkha_person_tr_1000,tr_1000_parks,tr_1000route,tr_1000Grid:Pentry,tr_1000_access
0,1006.157,271722.395,2685.0,57.958486,10.0,"(LINESTRING (90.4192166 23.8072591, 90.4193298...","[1271, 1273, 1274, 1382, 1383, 1384, 1513, 151...",1 high,7201.436,1.934060e+06,...,"[1100, 1101, 1102, 1270, 1271, 1272, 1273, 127...",1 high,23159.058,6.060704e+06,11304.0,54.224103,45.0,"(LINESTRING (90.4173969 23.8084543, 90.4177693...","[1098, 1099, 1100, 1101, 1102, 1269, 1270, 127...",1 high
1,2791.866,732496.517,6134.0,12.448537,23.0,"(LINESTRING (90.4136595 23.7996924, 90.4137228...","[2447, 2449, 2450, 2630, 2631, 2632, 2758, 275...",1 high,14555.490,3.875091e+06,...,"[2029, 2030, 2031, 2172, 2175, 2176, 2177, 230...",1 high,58705.616,1.506216e+07,40971.0,11.452871,166.0,"(LINESTRING (90.4129969 23.8048625, 90.4124451...","[1628, 1631, 1765, 1766, 1767, 1768, 1769, 177...",1 high
2,2286.774,446247.897,3707.0,12.143587,21.0,"(LINESTRING (90.4091151 23.803841, 90.4091765 ...","[1900, 1902, 2021, 2026, 2165, 2166, 2167, 216...",1 high,18135.066,3.488904e+06,...,"[1622, 1758, 1759, 1760, 1761, 1762, 1765, 176...",1 high,69988.634,1.444920e+07,39613.0,16.503392,179.0,"(LINESTRING (90.4138806 23.8056545, 90.4138777...","[1376, 1499, 1500, 1501, 1507, 1508, 1620, 162...",1 high
3,4321.771,1161472.547,7798.0,47.839788,29.0,"(LINESTRING (90.416775 23.8065858, 90.4164004 ...","[1379, 1380, 1510, 1511, 1512, 1633, 1634, 163...",1 high,21384.537,5.795854e+06,...,"[1271, 1379, 1380, 1382, 1383, 1384, 1510, 151...",1 high,67612.583,1.843988e+07,40218.0,45.550123,147.0,"(LINESTRING (90.4190122 23.8082482, 90.4205588...","[1100, 1101, 1269, 1270, 1271, 1272, 1273, 137...",1 high
4,2061.345,580583.554,5428.0,31.398276,19.0,"(LINESTRING (90.4147879 23.7939978, 90.4150593...","[3469, 3613, 3614, 3615, 3743, 3744, 3745, 374...",1 high,12934.760,3.745825e+06,...,"[3193, 3336, 3337, 3338, 3339, 3340, 3341, 346...",1 high,53797.167,1.603810e+07,41387.0,32.724175,139.0,"(LINESTRING (90.4159278 23.7977074, 90.415927 ...","[2908, 3040, 3041, 3042, 3044, 3188, 3190, 319...",1 high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1309.865,320203.553,2286.0,87.288511,9.0,"(LINESTRING (90.413408 23.7724674, 90.4134713 ...","[7135, 7136, 7137, 7255, 7256, 7257, 7258, 738...",1 high,8185.971,2.786793e+06,...,"[6685, 6815, 6878, 6879, 6882, 7001, 7002, 700...",1 high,26630.393,1.053604e+07,21075.0,144.542780,50.0,"(LINESTRING (90.4130678 23.7744734, 90.4128941...","[6685, 6686, 6687, 6688, 6814, 6815, 6878, 687...",1 high
164,1975.488,637978.304,5519.0,82.949846,17.0,"(LINESTRING (90.3869626 23.8728434, 90.3869658...","[23375, 23376, 23477, 23478, 23479, 23480, 235...",1 high,12460.633,4.072392e+06,...,"[23142, 23265, 23266, 23267, 23268, 23372, 233...",1 high,40205.264,1.311792e+07,26495.0,83.576276,81.0,"(LINESTRING (90.3862501 23.8742848, 90.3862521...","[23139, 23140, 23141, 23142, 23143, 23144, 232...",1 high
165,777.892,20014.947,284.0,83.393292,11.0,"(LINESTRING (90.3584268 23.845534, 90.358451 2...","[27046, 27047, 27142, 27143, 27144, 27145, 271...",2 medium,5749.057,1.496132e+05,...,"[26940, 26943, 26944, 27043, 27045, 27046, 270...",1 high,17833.843,4.776809e+05,964.0,88.964031,35.0,"(LINESTRING (90.3557967 23.8464271, 90.3558313...","[26940, 26941, 26942, 26943, 26944, 27043, 270...",1 high
166,1608.339,1215924.745,12390.0,69.759809,17.0,"(LINESTRING (90.431593 23.7408478, 90.4315992 ...","[13144, 13303, 13305, 13470, 13471, 13472, 134...",1 high,12307.398,9.327971e+06,...,"[12905, 12906, 12908, 13139, 13140, 13141, 131...",1 high,50776.501,3.950468e+07,101920.0,74.183385,130.0,"(LINESTRING (90.428532 23.7440093, 90.4285367 ...","[12430, 12431, 12594, 12595, 12596, 12597, 125...",1 high


In [49]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['dissolve_key'], how = 'outer')
            g_csv = g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')]
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_csv/')
                
            g_csv.to_csv('D:/Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                         +'m grids/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y','geometry_m','Grid_m_centroid'])], geometry = 'geometry_x', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_lines/')
                
            g_lines.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x','geometry_m','Grid_m_centroid'])], geometry = 'geometry_y', crs = 4326)
            
            if not os.path.exists('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/'):
                os.makedirs('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)+'m grids/Grid_pref_parks_geoms/')
                
            g_geoms.to_file('D:Dumps/Scores output WPL-OSM/'+str(grid_block_size)
                            +'m grids/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')
preference[0][0][0]

Dhaka Metropolitan
entrance
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/2)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/3)
park_prefer 300
park_prefer 600
park_prefer 1000
gravity**(1/5)
park_prefer 300
park_prefer 600
park_prefer 1000
all done


,level_0,index,geometry_x,G1-P0,real_grid,real_park,way_calc.,route_cost,steps,Grid_No,...,tr_600,poptr_600,adj_tr_600,walk_area_ha600,walkha_person600,tr_1000,poptr_1000,adj_tr_1000,walk_area_ha1000,walkha_person1000
4872,18788.0,18788.0,None,1.0,2.948429e+09,2.948429e+09,normal way,0.000,0.0,162,...,406.270,2437.620,406.270,16.667870,0.359974,806.270,4837.620,483.7620,16.667870,0.359974
5014,18789.0,18789.0,None,1.0,2.948429e+09,2.948429e+09,normal way,0.000,0.0,163,...,320.540,1923.240,320.540,16.667870,0.359974,720.540,4323.240,432.3240,16.667870,0.359974
8861,18790.0,18790.0,None,1.0,2.948429e+09,2.948429e+09,normal way,0.000,0.0,279,...,386.665,5413.310,386.665,16.667870,0.839939,786.665,11013.310,471.9990,16.667870,0.839939
8871,18791.0,18791.0,None,1.0,2.948429e+09,2.948429e+09,normal way,0.000,0.0,280,...,569.054,6828.648,569.054,16.667870,0.719948,969.054,11628.648,581.4324,16.667870,0.719948
8905,18735.0,18735.0,"LINESTRING (90.41282 23.81518, 90.41085 23.81511)",1.0,2.575844e+08,2.948429e+09,normal way,200.769,1.0,282,...,305.086,3966.118,305.086,16.667870,0.779944,705.086,9166.118,423.0516,16.667870,0.779944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10218,51293.0,51293.0,None,1.0,4.476488e+09,4.476488e+09,normal way,0.000,0.0,31123,...,389.698,141070.676,389.698,36.051869,10.041088,789.698,285870.676,473.8188,36.051869,10.041088
10221,51398.0,51398.0,"MULTILINESTRING ((90.35397 23.81767, 90.35404 ...",1.0,3.639410e+09,4.476488e+09,normal way,232.158,4.0,31126,...,301.665,374064.600,301.665,36.051869,34.394888,701.665,870064.600,420.9990,36.051869,34.394888
10242,49275.0,49275.0,"MULTILINESTRING ((90.38152 23.81740, 90.38150 ...",1.0,3.755414e+09,3.755411e+09,normal way,148.210,5.0,31159,...,405.872,33687.376,405.872,0.173789,477.591465,805.872,66887.376,483.5232,0.173789,477.591465
10244,49277.0,49277.0,"MULTILINESTRING ((90.38217 23.81744, 90.38215 ...",1.0,3.755414e+09,3.755411e+09,normal way,81.865,4.0,31160,...,434.853,33918.534,434.853,0.173789,448.820895,834.853,65118.534,500.9118,0.173789,448.820895


In [ ]:
print(round((time.time() - start) / 60,2),'mns')